## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Iquique,CL,-20.22,-70.14,62.60,72,90,17.22,overcast clouds
1,1,Avarua,CK,-21.21,-159.78,78.80,83,25,18.34,scattered clouds
2,2,Qaanaaq,GL,77.48,-69.36,11.07,78,0,6.96,clear sky
3,3,Ribeira Grande,PT,38.52,-28.70,69.78,81,87,10.27,overcast clouds
4,4,Novokruchininskiy,RU,51.78,113.77,37.40,82,40,4.90,scattered clouds


In [4]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature for your destination?"))
max_temp = float(input("What is the maximum temperature for your destination?"))

In [5]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Avarua,CK,-21.21,-159.78,78.80,83,25,18.34,scattered clouds
5,5,Sorong,ID,-0.88,131.25,78.78,87,100,8.84,overcast clouds
9,9,Varzea Da Palma,BR,-17.60,-44.73,83.16,31,42,6.55,scattered clouds
12,12,Vaini,TO,-21.20,-175.20,78.80,83,20,11.41,few clouds
13,13,Atuona,PF,-9.80,-139.03,78.15,77,17,20.02,light rain
18,18,Vila Velha,BR,-20.33,-40.29,80.01,83,2,14.99,clear sky
21,21,Davila,PH,18.47,120.58,77.00,100,75,2.24,broken clouds
23,23,Sao Filipe,CV,14.90,-24.50,79.32,80,10,7.92,clear sky
24,24,Panguna,PG,-6.32,155.48,79.47,75,100,2.17,light rain
25,25,Wanning,CN,18.80,110.40,78.96,88,61,3.80,heavy intensity rain


In [6]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [7]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()

In [8]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Avarua,CK,78.80,scattered clouds,-21.21,-159.78,
5,Sorong,ID,78.78,overcast clouds,-0.88,131.25,
9,Varzea Da Palma,BR,83.16,scattered clouds,-17.60,-44.73,
12,Vaini,TO,78.80,few clouds,-21.20,-175.20,
13,Atuona,PF,78.15,light rain,-9.80,-139.03,
18,Vila Velha,BR,80.01,clear sky,-20.33,-40.29,
21,Davila,PH,77.00,broken clouds,18.47,120.58,
23,Sao Filipe,CV,79.32,clear sky,14.90,-24.50,
24,Panguna,PG,79.47,light rain,-6.32,155.48,
25,Wanning,CN,78.96,heavy intensity rain,18.80,110.40,


In [9]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.") 

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [10]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.drop(hotel_df.loc[hotel_df['Hotel Name']==""].index)

In [11]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [12]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [13]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height=&#39;420px&#39;))